In [4]:
import numpy as np
import pandas as pd
from sklearn import metrics
from openpyxl import load_workbook
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestRegressor
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import multiprocessing
import os
import time
import gloce as gc
from osgeo import gdal
from osgeo import gdalconst
from mpl_toolkits.basemap import Basemap
from glob import glob
from math import ceil
import seaborn as sns
from scipy import stats
from scipy.stats import gaussian_kde
#return im_data, im_width, im_height, im_geotrans, im_proj
def read_img(filename):
    dt = gdal.Open(filename)
    im_width = dt.RasterXSize
    im_height = dt.RasterYSize
    im_bands = dt.RasterCount
    im_geotrans = dt.GetGeoTransform()
    im_proj = dt.GetProjection()
    im_data = dt.ReadAsArray(0,0,im_width,im_height)
    return im_data, im_width, im_height, im_geotrans, im_proj
def write_img(filename, im_proj, im_geotrans, im_data):
    if 'int8' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_UInt16
    else:
        datatype = gdal.GDT_Float32
    if len(im_data.shape) == 3:
        im_bands, im_height, im_width = im_data.shape
    else:
        im_bands, (im_height, im_width) = 1, im_data.shape
    driver = gdal.GetDriverByName("GTiff")
    dataset = driver.Create(filename, im_width, im_height, im_bands, datatype)
    dataset.SetGeoTransform(im_geotrans)
    dataset.SetProjection(im_proj)
    if im_bands == 1:
        dataset.GetRasterBand(1).WriteArray(im_data)
    else:
        for i in range(im_bands):
            dataset.GetRasterBand(i + 1).WriteArray(im_data[i])
spei_path='//weili/User2/mxsun/CESS_230818/SPEI_highRes/'
sm_path='//weili/User2/mxsun/CESS_230818/SPEI_highRes/RF/'
soil_path='//weili/User2/mxsun/CESS_230818/SPEI_highRes/RF/'
drv_path='//weili/User2/mxsun/CESS_230818/Pattern_drive/'
season_path='//weili/User2/mxsun/CESS_230818/Resistance_monthly/growth_season/'

In [2]:
"""
2012-2013三个map
start取最小值
end取最大值
"""

'\n2012-2013三个map\n'

# growth season data

In [6]:
gs_12_start=read_img(season_path+'VIPPHEN_EVI2_2012_start.tif')[0]
gs_13_start=read_img(season_path+'VIPPHEN_EVI2_2013_start.tif')[0]
gs_14_start=read_img(season_path+'VIPPHEN_EVI2_2014_start.tif')[0]

gs_12_end=read_img(season_path+'VIPPHEN_EVI2_2012_end.tif')[0]
gs_13_end=read_img(season_path+'VIPPHEN_EVI2_2013_end.tif')[0]
gs_14_end=read_img(season_path+'VIPPHEN_EVI2_2014_end.tif')[0]

np.min(gs_12_start),np.max(gs_12_start)

(-2, 710)

In [9]:
gs_12_start=gs_12_start.astype(np.float32)
gs_12_start[gs_12_start<=0]=np.nan
gs_13_start=gs_13_start.astype(np.float32)
gs_13_start[gs_13_start<=0]=np.nan
gs_14_start=gs_14_start.astype(np.float32)
gs_14_start[gs_14_start<=0]=np.nan

gs_12_end=gs_12_end.astype(np.float32)
gs_12_end[gs_12_end<=0]=np.nan
gs_13_end=gs_13_end.astype(np.float32)
gs_13_end[gs_13_end<=0]=np.nan
gs_14_end=gs_14_end.astype(np.float32)
gs_14_end[gs_14_end<=0]=np.nan

start=[gs_12_start,gs_13_start,gs_14_start]
end=[gs_12_end,gs_13_end,gs_14_end]
gs_start=np.nanmin(start,axis=0)
gs_end=np.nanmax(end,axis=0)

np.nanmin(gs_start),np.nanmax(gs_start),gs_start.shape

C:\Users\dess\AppData\Local\Temp\ipykernel_58872\3681516673.py:17: RuntimeWarning: All-NaN axis encountered
  gs_start=np.nanmin(start,axis=0)
C:\Users\dess\AppData\Local\Temp\ipykernel_58872\3681516673.py:18: RuntimeWarning: All-NaN axis encountered
  gs_end=np.nanmax(end,axis=0)


(1.0, 703.0, (3600, 7200))

In [10]:
#读取mask的坐标信息，然后warp drought数据
dt=read_img(season_path+'VIPPHEN_EVI2_2012_start.tif')
write_img(season_path+'VIPPHEN_EVI2_startday_240607.tif',dt[4],dt[3],gs_start)
write_img(season_path+'VIPPHEN_EVI2_endday_240607.tif',dt[4],dt[3],gs_end)

# convert from dayly to monthly

In [12]:
'''
#天转月，北半球start<end,南半球start>end
# original_matrix = np.random.randint(0, 366, size=(3600, 7200))  # 示例代码生成一个随机矩阵  

# 定义每个月的天数  
days_in_month = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])  
# 累计天数数组（从1开始，以匹配月份）  
cumulative_days = np.cumsum(days_in_month)  
# 创建一个与原始矩阵shape相同的空矩阵来存储结果  
month_matrix = np.zeros_like(gs_start, dtype=int)  
# 遍历原始矩阵的每个元素，并计算对应的月份  
for i in range(3600):
    for j in range(7200):  
        day_start = gs_start[i, j]
        # 找到day属于哪个月份（通过比较cumulative_days数组）  
        for k, cum_day in enumerate(cumulative_days):  
            if day_start < cum_day:  
                month_matrix[i, j] = k + 1  # k+1是因为cumulative_days是从0天开始累计的，而月份从1开始  
                break
        else:  
            # 如果day等于或大于一年的总天数，设置为12月（这里假设输入的day不会超过365）  
            month_matrix[i, j] = 12
np.min(month_matrix),np.max(month_matrix)
'''

(1, 12)

In [20]:
days_in_month = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])# 定义每个月的天数（不考虑闰年）  
# 计算累计天数（从0开始，因为searchsorted是从左侧边界开始搜索的）  
cumulative_days = np.cumsum(days_in_month) - 1  # 减1是为了与searchsorted的行为一致  
# 创建一个与原始矩阵shape相同的空矩阵来存储月份  
month_matrix = np.zeros_like(gs_start, dtype=int)  
# 使用searchsorted找到每个天数对应的月份索引（加1以转换为1-12月的表示）  
month_matrix = np.searchsorted(cumulative_days, gs_start, side='right') + 1  
# 注意：searchsorted的side='right'表示在找到的第一个大于或等于给定值的元素的索引处停止搜索  
# 由于我们的累计天数是从0开始的，并且searchsorted返回的是第一个大于给定值的索引，  
# 所以我们需要加1来得到正确的月份（从1开始）
# 如果原始矩阵中有大于365的值，上面的代码会将它们映射到13或更高的月份，但我们可以简单地设置它们为12月
month_matrix[month_matrix > 12] = 0
dt=read_img(season_path+'VIPPHEN_EVI2_2012_start.tif')
write_img(season_path+'VIPPHEN_EVI2_start-month_240607a.tif',dt[4],dt[3],month_matrix)

In [21]:
month_matrix = np.zeros_like(gs_end, dtype=int)  
month_matrix = np.searchsorted(cumulative_days, gs_end, side='right') + 1  
month_matrix[month_matrix > 12] = 0
write_img(season_path+'VIPPHEN_EVI2_end-month_240607a.tif',dt[4],dt[3],month_matrix)

In [23]:
def GetExtent(infile):
    ds = gdal.Open(infile)
    geotrans = ds.GetGeoTransform()
    xsize = ds.RasterXSize
    ysize = ds.RasterYSize
    min_x,max_y = geotrans[0],geotrans[3]
    max_x,min_y = geotrans[0]+xsize*geotrans[1],geotrans[3]+ysize*geotrans[5]
    ds = None
    return min_x,max_y,max_x,min_y
tt=GetExtent(spei_path+'csif_resistance_230321.tif')
gdal.Warp(season_path+'VIPPHEN_EVI2_end-month_240607_sub.tif',
          season_path+'VIPPHEN_EVI2_end-month_240607a.tif',
          dstSRS='EPSG:4326',
          outputBounds = [tt[0],tt[3],tt[2],tt[1]],
          xRes=0.01,
          yRes=0.01,
          targetAlignedPixels=True,
          creationOptions = ["COMPRESS=LZW"])

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000024BF4BFD7A0> >